**1. Wstęp**

W ramch projketu opracowalśmy model regresyjny do prognozowania cen wynajmu mieszkań w Poznaniu. Zastosowaliśmy biblioteki LightGBM oraz XGBoost wsparte imputacją braków danych (KNN) oraz transformacją logarytmiczną zmiennej celu. Nasza grupa na Kaggle nazywała się Martyna & Piotr (#3) z wynikiem 78508.79 uzyskanym LightGBM. 

**2. Metodyka**

2.1 Przygotowanie danych zaczeliśmy od standaryzacji i oczyszczenia zbioru z błędnych wartości. 
 - Zidentyfikowaliśmy wartości -999 oraz -9, które w zbiorze służyły jako oznaczenie braków danych. Zostały one zmienone na NAN, aby umożliwić poprawną imputację.
 - Stworzyliśmy pętlę weryfikującą poprawność danych numerycznych, gdzie wszytskie wartości ujemne zmienione zostały na brak danych.

2.2 Następnie zastosowalismy dwuetapową imputcję danych.
- Na początku odzyskaliśmy dane z tytułów ogłoszeń. Stworzyliśmy funkcję opartą na wyrażeniach reguralnych, które wyodrębniały z tekstu np. powierzchnię. Pozwoliło nam to uzupełnić 400 braków w flat_area, 1531 w flat_rooms oraz 569 w quarter.
- Dla pozostałych danych numerycznych oraz kategorycznych, których nie udało się odzyskać zastosowano algorytmy.
    - Zmienne numeryczne uzupełniono za pomocą KNN, gdzie braki uzupełniano średnią wartością od 5 najbardziej podobnych sąsiadów.
    - Dla zmiennych binarnych zastosowano imputację opartą na regresji logistycznej, która dla każdej cechy trenowała oddzielny model klasyfikacyjny. Braki przewidywano na podsatwie parametrów fizycznych, a w przypadkach, w których model nie mógł zadziałać stosowano modę.
    - Do imputacji zmiennej quarter opracowano dedykowaną procedurę imputacji opartą na KNN. Dzielnice zostały zakodowane numerycznie, a brakujące wartości oszacowano na podstawie 5 najbardziej podobnych ofert.

2.3 Przed trenowaniem modeli dokonaliśmy analizy rozkładu zmiennej celu (price), która wykazała silną skośność prawostronną (drogie apartamenty jako wartości odstające). Aby poprawić stabilnośc procesu oraz zminimalizować wpływ outlinerów zastosowalismy transformację logistyczną. W implementacji wykorzystano wrapper TransformedTargetRegressor z biblioteki Scikit-learn. Pozwoliło to na automatyczne logarytmowanie danych treningowych oraz przekształcenie predykcji do oryginalnej skali cenowej, co uprościło przetwarzanie oraz ewaluacje.

2.4 W etapie modelowania zastosowaliśmy algorytmy oparte na wzmacnianiu gradientowym, w tym XGBoost oraz LightGBM w różnych konfiguracjach. Ostatecznie najlepsze rezultaty osiągnięto przy użyciu LightGBM bez logarytmicznej transformacji zmiennej celu. Model ten wykazał się wysoką skutecznością w wyłapywaniu nieliniowych zależności między cechami, a ceną wynajmu. Zwycięski algorytm Gradient Boosting Decision Tree miał kluczowe parametry wybrane drogą eksperymentalną. 

**3. Wyniki**

3.1 W trakcie prac nad projketem przetestowliśmy cztery główne konfiguracje modeli, gdzie miarą błędu było RMSE. Najlepsze uzyskane wyniki z każdego moelu przedstawia tabela poniżej. 

| Model | Konfiguracja | Wynik (RMSE) | Komentarz |
| :--- | :--- | :--- | :--- |
| **XGBoost** | Baseline, domyślne parametry | **80121.76** | Dobry punkt wyjścia. |
| **XGBoost_log** | Transformacja logarytmiczna celu | **81842.94** | Zastosowanie logarytmizacji pogorszyła wynik. |
| **LightGBM_log** | Transformacja logarytmiczna celu | **81987.70** | Transformacja również nie przyniosła poprawy. |
| **LightGBM** | **Brak transformacji, learning_rate=0.05** | **78508.79** | Odpowiednie dostrojenie dało najmniejszy błąd predykcji |

3.2 Proces dochodzenia do najlepszego wyniku przebiegał wieloetapowo.
- Baseline: Uruchomienie modelu XGBoost na domyślnych parametrach pozwolił ustalić punkt odniesienia z wynikiem RMSE na poziomie 80121. Był to dobry start, który wskazywał możliwość dalszej optymalizacji. 
- Zastosowanie transformacji: W celu zniwelowania skośności rozkładu cen zastosowaliśmy transformację logarytymiczną. Ku naszemu zaskoczeniu zabieg ten pogorszył wynik dla obu testowanych przez nas algorytmów. Sugeruje nam to, że błędy estymacji popełniane na skali logarytmicznej, po powrotym prszekształceniu uległy wzmocnieniu. Wynika z tego, że transformacja okazała się niekorzystna.
- Zmiana na LightGBM i optymalizacja: Mimo braku poprawy po transformacji danych zmiana algorytmu na LightGBM na surowych danych pzyniosła poprawę. Dzięki szybszemu procesowi uczenia mogliśmy przetestować większy zakres hiperparametrów.
- Ostateczna konfiguracja: Odpowiednio wyregulowany model gradientowy potrafił samodzielnie i skutecznie zamodelować dane surowe, bez konieczności stosowania transformacji matematycznych. Dzięki temu model LightGBM uzyskał wynik o ponad 1000pkt lepszy od baseline'u.

**4. Podsumowanie**

Realizacja projektu dostarczyła kilku kluczowych informacji. Największym zaskoczeniem był fakt, że transformacja logarytmiczna zmiennej celu pogorszyła wyniki. Nauczyło nas to, że teoretyczne założenia (o normalizacji rozkładu cen) należy zawsze weryfikować eksperymentalnie. Dobrze dostrojony model na surowych danych (LightGBM) poradził sobie lepiej z wartościami odstającymi niż model z wymuszoną transformacją. Dodatkowo kluczem do sukcesu nie był sam algorytm, ale przygotowanie danych. "Uratowanie" wielu rekordów poprzez wyciągnięcie ichz tytułów ogłoszeń dało modelowi więcej informacji niż jakakolwiek zmiana hiperparametrów. Co prowadzi nas do wniosków, że prawidłowa imputacja danych jest kluczowa do stworzenia dobrego modelu. Uzupełnienie ich za pomocą Regex czy KNN, zamiast usuwania braków pozwoliło nam zachować wiele rekordów co przy modelowaniu ceny było bardzo pomocne.

**5. Kod do odtworzenia wyników**